In [60]:
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from ast import literal_eval
from sklearn.model_selection import train_test_split

import os
import random
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) # magic number :)

### 학습 데이터 불러오기

In [3]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [58]:
# 데이터셋 로드
test_df = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
test_df.info()

# Flatten the JSON dataset
records = []
for _, row in test_df.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
test_df = pd.DataFrame(records)
test_df["choices_len"] = test_df["choices"].apply(len)

test_dataset = Dataset.from_pandas(test_df)

# Model load


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     869 non-null    int64 
 1   id             869 non-null    object
 2   paragraph      869 non-null    object
 3   problems       869 non-null    object
 4   question_plus  44 non-null     object
dtypes: int64(1), object(4)
memory usage: 34.1+ KB


In [4]:
# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2031 non-null   object
 1   paragraph      2031 non-null   object
 2   question       2031 non-null   object
 3   choices        2031 non-null   object
 4   answer         2031 non-null   int64 
 5   question_plus  0 non-null      object
dtypes: int64(1), object(5)
memory usage: 95.3+ KB


In [6]:
df.head()

,id,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,None
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,None
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,None


#### Data Preprocessing

In [7]:
df["choices_len"] = df["choices"].apply(len)
df['choices_len'].value_counts(dropna=False)

choices_len
5    1239
4     792
Name: count, dtype: int64

In [ ]:
train_df, valid_df = train_test_split(
    df,
    test_size=0.15,        
    random_state=42,
    stratify=df["choices_len"]
)

In [9]:
train_df["choices_len"].value_counts(normalize=True)
valid_df["choices_len"].value_counts(normalize=True)

choices_len
5    0.609836
4    0.390164
Name: proportion, dtype: float64

In [10]:
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))

dataset = DatasetDict({
    "train": train_ds,
    "validation": valid_ds
})

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len'],
        num_rows: 1726
    })
    validation: Dataset({
        features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len'],
        num_rows: 305
    })
})

### Tokenizer Load
- 모델은 굳이 필요 없음.

In [12]:
MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B"
    )
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-8B",
    torch_dtype=torch.float16,
    device_map="auto",
    )

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

### Message 생성

In [13]:
# prompt 생성
# message 구조로 감싸기
# chat_template로 실제 모델 입력 문자열 생성
# tokenizer로 input_ids/ attention_mask 생성
# Trainer에 넘길 dataset 완성

In [14]:
### 정책별 System Prompt 함수
def get_system_message(row, system_prompts, prompt_policy):
    """
    row: 하나의 데이터 행
    system_prompts: {choices_len: {version: prompt_text}}
    prompt_policy: {choices_len: version}
    """
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    return system_prompts[choices_len][version]

In [15]:
# System Prompt 관리 구조 설명

# SYSTEM_PROMPTS:
# - 실제 system prompt 문구들을 모두 정의해 두는 "저장소"
# - 문제 유형(choices_len = 4 / 5)별로 나누고, 각 유형 안에서 v1, v2 처럼 여러 버전의 prompt를 보관
# - 즉, 여기에는 "쓸 수 있는 모든 프롬프트 후보"가 들어 있다.
#
# 예:
# SYSTEM_PROMPTS = {
#     4: { "v1": "...", "v2": "..." },
#     5: { "v1": "...", "v2": "..." }
# }

SYSTEM_PROMPT_4_V1 = (
    "당신은 **지식 추론(Knowledge Inference) 전문가**입니다. "
    "이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다. "
    "지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 "
    "가장 타당한 선택지 하나를 고르십시오."
)

SYSTEM_PROMPT_5_V1 = (
    "당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다. "
    "이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다. "
    "당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.\n\n"
)

# PROMPT_POLICY:
# - 이번 실험(run)에서 "어떤 system prompt 버전을 사용할지"를 결정하는 설정값
# - choices_len(4 or 5) → 사용할 prompt 버전(v1, v2, ...)
# - 실험을 바꿀 때는 이 딕셔너리만 수정

SYSTEM_PROMPTS = {
    4: {
        "v1": SYSTEM_PROMPT_4_V1,
    },
    5: {
        "v1": SYSTEM_PROMPT_5_V1,
    }
}

SYSTEM_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [16]:
print(SYSTEM_PROMPTS[4]['v1'])

당신은 **지식 추론(Knowledge Inference) 전문가**입니다. 이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다. 지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 가장 타당한 선택지 하나를 고르십시오.


In [17]:
### 정책별 User Prompt 함수
def get_user_message(row, user_prompts, prompt_policy):
    """
    row: 데이터 행
    user_prompts: 템플릿 저장소
    prompt_policy: 버전 정책
    """
    # 메타 데이터 확인
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    
    # 해당 버전의 템플릿 세트 가져오기 (plus, no_plus가 들어있음)
    template_set = user_prompts[choices_len][version]
    
    # 데이터 준비
    paragraph = row['paragraph']
    question = row['question']
    choices_str = "\n".join([f"{i+1}. {c}" for i, c in enumerate(row['choices'])])
    q_plus = row.get('question_plus', None)
    
    # 분기 처리 및 포맷팅 (여기가 핵심!)
    # q_plus가 존재하고, nan이 아닐 때 -> Plus 템플릿 사용
    if q_plus and str(q_plus) != 'nan':
        return template_set["plus"].format(
            paragraph=paragraph,
            question_plus=q_plus, # 여기 들어감
            question=question,
            choices=choices_str
        )
    # q_plus가 없을 때 -> No Plus 템플릿 사용
    else:
        return template_set["no_plus"].format(
            paragraph=paragraph,
            question=question,
            choices=choices_str
        )

In [18]:
# =========================
# User Prompt Templates (V1)
# =========================

# 4지선다 + <보기> 있음
USER_PROMPT_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 4지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 5지선다 + <보기> 있음
USER_PROMPT_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 5지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

USER_PROMPTS = {
    4: {
        "v1": {
            "plus": USER_PROMPT_PLUS_4_V1,
            "no_plus": USER_PROMPT_NO_PLUS_4_V1,
        },
        # "v2": {...}
    },
    5: {
        "v1": {
            "plus": USER_PROMPT_PLUS_5_V1,
            "no_plus": USER_PROMPT_NO_PLUS_5_V1,
        },
        # "v2": {...}
    }
}

USER_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [19]:
### 형식 확인
import random

train_ds = dataset["train"]

# 1) 랜덤 샘플 1개 뽑기
idx = random.randrange(len(train_ds))
row = train_ds[idx]

print("idx:", idx)
print("id:", row["id"])
print("choices_len:", row["choices_len"])
print("question_plus:", row["question_plus"])

# 2) system/user 메시지 만들기
sys_msg = get_system_message(row, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
user_msg = get_user_message(row, USER_PROMPTS, USER_PROMPT_POLICY)

print("\n===== SYSTEM =====")
print(sys_msg)

print("\n===== USER =====")
print(user_msg)
print("\n... (len:", len(user_msg), ")")

# 3) messages 구성 (정답은 숫자만)
messages = [
    {"role": "system", "content": sys_msg},
    {"role": "user", "content": user_msg},
    {"role": "assistant", "content": str(row["answer"])},
]

# 4) chat template 적용
text = tokenizer.apply_chat_template(messages, tokenize=False)

print("\n===== CHAT TEMPLATE =====")
print(text)

idx: 1309
id: generation-for-nlp-1957
choices_len: 5
question_plus: None

===== SYSTEM =====
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다. 이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다. 당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.



===== USER =====
### 지문
건강분야 연구개발(R&D)에 대한 정부의 투자가 급격히 늘어나고 있다. 하지만 실제 사업화로 연결된 과제는 100개 중 6개도 채 되지 않는 것으로 조사됐다. 수요 조사를 정확하게 하지 않은 상태에서 마구잡이로 연구과제를 선정하는 경우가 많기 때문이다. 일단 받은 예산으로 개발부터 하고 보자는 식의 R&D 문화가 확산돼 있는 것도 원인이다.○리포트로만 남는 국민 세금11일 보건복지부와 과학기술정책연구원, 한국보건산업진흥원 등에 따르면 일상적 건강영역에 대한 정부 R&D 투자액은 2009년 679억원에서 지난해 1178억원으로 늘었다. 과제 수도 444개에서 812개로 증가했다. 만성질환자가 증가하고 건강관리에 대한 관심이 높아진 상황이 반영된 것이다.일상적 건강영역 R&D는 전체 건강 R&D 중 중증질환 치료와 직결된 것을 제외한 것으로 재활, 건강식품, 의료보조기기, 만성질환 연구 등이 모두 포함된다.그러나 이 분야의 정부 R&D 과제 중 실제 사업화로 이어진 과제는 2009년 33개에서 2012년 46개로 늘어나는 데 그쳐 사업화율은 오히려 7.43%에서 5.85%로 떨어졌다. 전체 사업화 건수(하나의 연구에 중복 사업화 포함)도 82건에서 62건으로 줄었다. 고용창출 인원 수도 같은 기간 283명에서 172명으로 감소했다.한 대학 연구소 관계자는 “국가가 지원한 건강관련 R&D 사업의 6% 정도만 사업화로 이어지는 것은 심각하게 봐야 한다”며 “많은 세금을 쏟아부어 개발한 기술과 제품, 서비스 등이 실제 사업으로 이어지지 않고 아무도

In [20]:
# 공통 Assistant Prompt 함수
def get_assistant_message(row):
    """
    Assistant 메시지 생성 함수.
    Qwen3 모델의 토크나이저 템플릿이 자동으로 <think> 태그를 처리하므로,
    여기서는 순수한 정답(Label) 텍스트만 반환
    """
    return str(row['answer'])

In [21]:
def build_messages(example):
    """
    원본 example(row)로부터 학습용 chat messages를 구성한다.
    - choices_len(4/5) 및 question_plus 유무에 따라 system/user 프롬프트를 선택
    - assistant는 정답 숫자만
    - 이후 평가/추적용으로 id, label도 함께 유지
    """
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)
    asst_msg = get_assistant_message(example)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": asst_msg},
        ],
        "label": int(example["answer"]),
    }

In [22]:
build_messages(dataset['train'][24])

{'id': 'generation-for-nlp-1948',
 'messages': [{'role': 'system',
   'content': '당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다. 이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다. 당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.\n\n'},
  {'role': 'user',
   'content': '### 지문\n국제축구연맹(FIFA) 뇌물 스캔들 파문이 일파만파 번지고 있다. 미국 법무부의 조사 착수 배경을 놓고 미국과 러시아가 갈등을 빚고 있고, FIFA와 거래해온 대형 금융회사들로까지 조사가 확대될 조짐을 보이고 있다.외신들에 따르면 블라디미르 푸틴 러시아 대통령은 28일(현지시간) 한 언론과의 인터뷰에서 미 법무부의 FIFA 수사와 관련, “누군가가 무언가를 위반했을 것으로 추정할 수 있지만 미국이 이와 무슨 상관이란 말인가”라며 “이번 수사는 국제기구의 운영 원칙에 대한 심각한 침해”라고 목소리를 높였다. 그는 “미국이 제프 블래터 FIFA 회장(사진)의 재선을 막으려 한다는 것은 의심의 여지가 없다”며 “이번 조사가 러시아의 2018년 월드컵 개최에 아무런 영향을 미치지 않아야 한다”고 강조했다. 블래터 회장은 2018년 월드컵 개최지 선정 때 공개적으로 러시아를 지지했다.뉴욕타임스(NYT)는 우크라이나 사태 등으로 러시아와 불편한 관계에 있는 미국이 이번 수사를 통해 러시아를 압박하려는 게 아니냐는 의혹이 일면서 FIFA 뇌물 스캔들이 양국 외교 갈등으로 비화할 조짐을 보이고 있다고 보도했다. 푸틴이 이번 사건을 미국의 러시아 월드컵 개최 저지를 위한 사전 공작쯤으로 해석하고 있다는 것이다.미국은 정치적 의도를 부인했다. 제프리 래스키 미 국무부 대변인은 “이번 수사엔 정치적 목적이 전혀 없으며 부패 행위는 용납할 수 없다는 미국의 명확한 메시지를 보여주는 것일 뿐”이라고 말했다. 래스키 대변인은 “FIFA 간부들에

In [23]:
def to_text(example):
    """
    messages(list[dict])를 tokenizer의 chat_template 규칙에 따라
    단일 텍스트로 직렬화한다.
    """
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False,  
    )
    return {"text": text}

In [24]:
def tokenize_fn(example, truncation=True, max_length=2048, padding=False):
    """
    batched=True면 example["text"]는 List[str]
    batched=False면 example["text"]는 str
    """
    tok_kwargs = dict(truncation=truncation, padding=padding)
    if truncation is True:
        tok_kwargs["max_length"] = max_length

    out = tokenizer(example["text"], **tok_kwargs)
    
    return {
        "input_ids": out["input_ids"],
        "attention_mask": out["attention_mask"],
    }


In [25]:
orig_cols = dataset["train"].column_names
print(orig_cols)

['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len']


In [26]:
dataset_msg = dataset.map(
    build_messages,
    batched=False,
    remove_columns=orig_cols,
    desc="Build messages",
)

Build messages:   0%|          | 0/1726 [00:00<?, ? examples/s]

Build messages:   0%|          | 0/305 [00:00<?, ? examples/s]

In [27]:
dataset_msg

DatasetDict({
    train: Dataset({
        features: ['id', 'messages', 'label'],
        num_rows: 1726
    })
    validation: Dataset({
        features: ['id', 'messages', 'label'],
        num_rows: 305
    })
})

In [28]:
dataset_text = dataset_msg.map(
    to_text,
    batched=False,
    remove_columns=["messages"],
    desc="Serialize to text",
)

Serialize to text:   0%|          | 0/1726 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/305 [00:00<?, ? examples/s]

In [29]:
tokenized_dataset = dataset_text.map(
    tokenize_fn,
    batched=True,
    num_proc=4, 
    remove_columns=["text"],
    load_from_cache_file=True,
    desc="Tokenizing",
)

Tokenizing (num_proc=4):   0%|          | 0/1726 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/305 [00:00<?, ? examples/s]

In [30]:
### 전체 pipeline 다시 확인
orig_cols = dataset["train"].column_names
dataset_msg = dataset.map(
    build_messages,
    batched=False,
    remove_columns=orig_cols,
    desc="Build messages",
)
dataset_text = dataset_msg.map(
    to_text,
    batched=False,
    remove_columns=["messages"],
    desc="Serialize to text",
)
tokenized_dataset = dataset_text.map(
    tokenize_fn,
    batched=True,
    fn_kwargs={"truncation": True, "max_length": 2048, "padding": False},
    num_proc=4, 
    remove_columns=["text"],
    load_from_cache_file=True,
    keep_in_memory=True,
    desc="Tokenizing",
)

Build messages:   0%|          | 0/1726 [00:00<?, ? examples/s]

Build messages:   0%|          | 0/305 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/1726 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/305 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/1726 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/305 [00:00<?, ? examples/s]

In [31]:
print("\n=== 변환 완료 ===")
print("Train 개수:", len(tokenized_dataset["train"]))
print("첫 번째 샘플 Keys:", tokenized_dataset["train"][0].keys())


=== 변환 완료 ===
Train 개수: 1726
첫 번째 샘플 Keys: dict_keys(['id', 'label', 'input_ids', 'attention_mask'])


In [ ]:
split = "train"  # 또는 "validation"

msg_ds = dataset_msg[split]
text_ds = dataset_text[split]
tok_ds  = tokenized_dataset[split]

idx = random.randrange(len(msg_ds))

print("\n--- messages example ---")
ex_msg = msg_ds[idx]
print("id:", ex_msg["id"])
print("label:", ex_msg["label"])

for j, m in enumerate(ex_msg["messages"]): 
    print(f"[{j}] role={m['role']}")
    print(m["content"], "...\n") 

print("\n--- text example ---")
ex_text = text_ds[idx]["text"]
print(ex_text, "...\n")

print("\n--- tokenized example ---")
ex_tok = tok_ds[idx]
print("len:", len(ex_tok["input_ids"]))
print("decoded preview:")
print(tokenizer.decode(ex_tok["input_ids"], skip_special_tokens=False))


--- messages example ---
id: generation-for-nlp-1209
label: 2
[0] role=system
당신은 **지식 추론(Knowledge Inference) 전문가**입니다. 이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다. 지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 가장 타당한 선택지 하나를 고르십시오. ...

[1] role=user
### 지문
저는 순응하는 마음으로 그리스도교 개혁에 관한 몇 가지 지점들을 취합해 독일 국가의 그리스도인 귀족 앞에 그대로 제시하고자 합니다. 미천한 개인에 불과한 제가 귀족 분들께 이렇게 나서서 말씀드리는 것은 단순한 오만이나 고집 때문이 아닙니다. 모든 그리스도교, 특히 독일의 그리스도교를 억압하는 고통과 비참함으로 인해 저를 비롯한 모든 사람은 울부짖으며 도움을 요청하게 되었습니다… “이 로마 가톨릭 교도들은 자기들 주위로 세 개의 벽을 쌓아 스스로를 보호함으로써 그 누구도 그들을 개혁하지 못했습니다. 이로 인해 모든 기독교권이 크게 추락했습니다… 세속의 권력은 영성에 어떠한 권한도 없다… 성경을 해석할 수 있는 자는 교황이 유일하다… 공의회를 소집할 수 있는 자는 교황이 유일하다… 이제 공의회에서 다루어야 할 문제, 교황과 추기경과 주교와 모든 학식 있는 자가 밤낮으로 골몰해야 할 문제들을 생각해봅시다… 그리스도의 대리자이자 성 베드로의 후계자라고 뽐내는 그리스도교의 수장이 어떠한 왕이나 황제도 따라올 수 없는 세속적 허세 속에 사는 모습을 지켜보는 일은 비통하고 끔찍한 일이 아닐 수 없습니다. 그리스도교에서 ‘추기경’이라고 불리는 사람들은 왜 있는 것입니까? 제가 말씀드리지요. 이탈리아와 독일에는 부유한 수녀회, 기금, 영지, 성직급이 있었는데, 이것들을 로마의 손에 가장 손쉽게 쥐어주기 위해 추기경이란 자리를 만들고 이 추기경에게 주교 관구, 수녀회, 고위 성직

In [33]:
print(print(tokenizer.decode(ex_tok["input_ids"], skip_special_tokens=True)))

system
당신은 **지식 추론(Knowledge Inference) 전문가**입니다. 이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다. 지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 가장 타당한 선택지 하나를 고르십시오.
user
### 지문
저는 순응하는 마음으로 그리스도교 개혁에 관한 몇 가지 지점들을 취합해 독일 국가의 그리스도인 귀족 앞에 그대로 제시하고자 합니다. 미천한 개인에 불과한 제가 귀족 분들께 이렇게 나서서 말씀드리는 것은 단순한 오만이나 고집 때문이 아닙니다. 모든 그리스도교, 특히 독일의 그리스도교를 억압하는 고통과 비참함으로 인해 저를 비롯한 모든 사람은 울부짖으며 도움을 요청하게 되었습니다… “이 로마 가톨릭 교도들은 자기들 주위로 세 개의 벽을 쌓아 스스로를 보호함으로써 그 누구도 그들을 개혁하지 못했습니다. 이로 인해 모든 기독교권이 크게 추락했습니다… 세속의 권력은 영성에 어떠한 권한도 없다… 성경을 해석할 수 있는 자는 교황이 유일하다… 공의회를 소집할 수 있는 자는 교황이 유일하다… 이제 공의회에서 다루어야 할 문제, 교황과 추기경과 주교와 모든 학식 있는 자가 밤낮으로 골몰해야 할 문제들을 생각해봅시다… 그리스도의 대리자이자 성 베드로의 후계자라고 뽐내는 그리스도교의 수장이 어떠한 왕이나 황제도 따라올 수 없는 세속적 허세 속에 사는 모습을 지켜보는 일은 비통하고 끔찍한 일이 아닐 수 없습니다. 그리스도교에서 ‘추기경’이라고 불리는 사람들은 왜 있는 것입니까? 제가 말씀드리지요. 이탈리아와 독일에는 부유한 수녀회, 기금, 영지, 성직급이 있었는데, 이것들을 로마의 손에 가장 손쉽게 쥐어주기 위해 추기경이란 자리를 만들고 이 추기경에게 주교 관구, 수녀회, 고위 성직자들을 넘겼습니다. 그리고 그 결과 하느님을 섬기는 일이 망가지고 말았습니다.

### 질문
위 글의 마틴 루터가 밝힌 견해와 유사한 견해를 표명한 개혁가

In [34]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1726
    })
    validation: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 305
    })
})

In [35]:
### 토큰화된 길이 확인
ds = tokenized_dataset["train"]

lengths = [len(x) for x in ds["input_ids"]]

print("count:", len(lengths))
print("min:", min(lengths))
print("max:", max(lengths))
print("mean:", np.mean(lengths))
print("p50:", np.percentile(lengths, 50))
print("p75:", np.percentile(lengths, 75))
print("p90:", np.percentile(lengths, 90))
print("p95:", np.percentile(lengths, 95))
print("p99:", np.percentile(lengths, 99))

count: 1726
min: 309
max: 1865
mean: 887.3823870220162
p50: 868.0
p75: 1097.75
p90: 1384.5
p95: 1527.0
p99: 1662.0


In [36]:
tokenizer.chat_template

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0].role == \'system\' %}\n        {{- messages[0].content + \'\\n\\n\' }}\n    {%- endif %}\n    {{- "# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0].role == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0].content + \'<|im_end|>\\n\' }}\n    {%- endif %}\n{%- endif %}\n{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}\n{%- for message in messa

In [37]:
response_template = "<|im_start|>assistant\n"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

In [38]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1726
    })
    validation: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 305
    })
})

In [39]:
# train_ds는 id/label 제거한 상태 OK
train_ds = tokenized_dataset["train"].remove_columns(["id", "label"])

# 길이가 다른 샘플 2개 뽑기 (패딩을 보려면 반드시 길이 차이가 나야 함)
i1, i2 = 0, 1
print("len1:", len(train_ds[i1]["input_ids"]))
print("len2:", len(train_ds[i2]["input_ids"]))

sample_batch = [train_ds[i1], train_ds[i2]]

# collate
collated = data_collator(sample_batch)

# 텐서 shape 확인 (= 배치 단위로 같은 길이로 패딩 됐는지)
print("input_ids shape:", collated["input_ids"].shape)        # (B, L)
print("attention_mask shape:", collated["attention_mask"].shape)
print("labels shape:", collated["labels"].shape)

# 패딩 토큰 개수 확인 (pad_token_id 기준)
pad_id = tokenizer.pad_token_id
print("pad_token_id:", pad_id)

for b in range(collated["input_ids"].shape[0]):
    num_pad = (collated["input_ids"][b] == pad_id).sum().item() if pad_id is not None else 0
    real_len = collated["attention_mask"][b].sum().item()
    print(f"[batch {b}] real_len(attn=1):", real_len, "| num_pad:", num_pad)

# labels 마스킹 확인: -100이 '학습 제외' 영역
for b in range(collated["labels"].shape[0]):
    num_ignored = (collated["labels"][b] == -100).sum().item()
    print(f"[batch {b}] ignored(-100):", num_ignored)

# 눈으로 보고 싶으면 끝부분 몇 토큰 decode
b = 0
tail_ids = collated["input_ids"][b][-80:].tolist()
tail_lbl = collated["labels"][b][-80:].tolist()

print("\n--- decoded tail (input_ids) ---")
print(tokenizer.decode(tail_ids, skip_special_tokens=False))

print("\n--- tail labels (show -100 positions) ---")
print(tail_lbl)

len1: 855
len2: 323
input_ids shape: torch.Size([2, 855])
attention_mask shape: torch.Size([2, 855])
labels shape: torch.Size([2, 855])
pad_token_id: 151643
[batch 0] real_len(attn=1): 855 | num_pad: 0
[batch 1] real_len(attn=1): 323 | num_pad: 532
[batch 0] ignored(-100): 848
[batch 1] ignored(-100): 848

--- decoded tail (input_ids) ---
사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:<|im_end|>
<|im_start|>assistant
<think>

</think>

4<|im_end|>


--- tail labels (show -100 positions) ---
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [40]:
# assistant 턴 시작 지점이 labels에서 -100이 아닌 첫 위치인가?
first_real = (collated["labels"][0] != -100).nonzero()[0].item()
print("첫 학습 토큰 위치:", first_real)
print("해당 토큰:", tokenizer.decode([collated["input_ids"][0][first_real]]))
# <think> 또는 <|im_start|>assistant

# -100 개수가 입력(user) 길이와 일치하는가?
# 848개가 -100 → user prompt + system이 대략 이 정도 길이

# 패딩 토큰도 -100인가?
if pad_id is not None:
    pad_positions = (collated["input_ids"][1] == pad_id)
    labels_at_pad = collated["labels"][1][pad_positions]
    assert (labels_at_pad == -100).all(), "패딩 위치의 labels가 -100이어야 함"
    print("패딩 영역도 올바르게 -100 처리됨")

첫 학습 토큰 위치: 848
해당 토큰: <think>
패딩 영역도 올바르게 -100 처리됨


### Metric 함수 구현

본 학습에서는 모델의 성능을 검증하기 위해 **Accuracy(정확도)**와 **Macro-F1 Score**를 평가지표로 사용
모델의 출력은 항상 고정된 7개의 토큰 시퀀스를 가지며, 실제 정답(1~5)은 **뒤에서 3번째(Index -3)**에 위치

#### 1. 모델 출력 구조 분석 (7 Tokens)
학습 데이터는 다음과 같은 구조로 구성되어 있으며, 평가 시 **Index -3**의 로짓(Logits)만을 사용하여 정답을 판별

| 순서 (Index) | 역순 (Neg Index) | Token ID | 내용 (Decoded) | 역할 | 비고 |
|:---:|:---:|:---:|:---:|:---|:---|
| 0 | -7 | 151667 | `<think>` | 생각 시작 | |
| 1 | -6 | 271 | `\n` | 줄바꿈 | Qwen 전용 줄바꿈 |
| 2 | -5 | 151668 | `</think>` | 생각 끝 | 내용 없음 |
| 3 | -4 | 271 | `\n` | 줄바꿈 | |
| **4** | **-3** | **19 (가변)** | **정답 숫자** | **실제 정답** | **Target (채점 대상)** |
| 5 | -2 | 151645 | `<|im_end|>` | 턴 종료 | |
| 6 | -1 | 198 | `\n` | 문장 끝 | 패딩 직전 구분자 |

<br>

#### 2. 평가 지표 (Metrics)

* **Accuracy (정확도)**
    * 전체 검증 샘플 중에서 모델이 정답을 정확히 맞춘 비율을 의미
    * 가장 직관적인 지표로, 전체적인 성능을 파악하는 데 사용

* **Macro-F1 Score**
    * 정답 선지(1~5번)를 각각 독립된 클래스로 간주하여 클래스별 F1 Score를 계산한 뒤, 단순 평균(Arithmetic Mean)을 낸 값
    * 특정 번호에 정답이 쏠려 있을 경우 발생할 수 있는 편향을 방지하고, 모든 선지를 고르게 잘 맞추는지 평가하기 위해 사용

In [41]:
### Qwen3 1,2,3,4,5 토큰 id 확인
# 가장 확실: encode 결과 보기
for s in ["1","2","3","4","5"," 1"," 2"," 3"," 4"," 5"]:
    ids = tokenizer.encode(s, add_special_tokens=False)
    toks = tokenizer.convert_ids_to_tokens(ids)
    print(f"{s!r} -> ids={ids}, toks={toks}")

# 단일 토큰 매핑을 기대한다면 (토크나이저에 따라 None/-1일 수 있음)
for tok in ["1","2","3","4","5"]:
    print(tok, "->", tokenizer.convert_tokens_to_ids(tok))

'1' -> ids=[16], toks=['1']
'2' -> ids=[17], toks=['2']
'3' -> ids=[18], toks=['3']
'4' -> ids=[19], toks=['4']
'5' -> ids=[20], toks=['5']
' 1' -> ids=[220, 16], toks=['Ġ', '1']
' 2' -> ids=[220, 17], toks=['Ġ', '2']
' 3' -> ids=[220, 18], toks=['Ġ', '3']
' 4' -> ids=[220, 19], toks=['Ġ', '4']
' 5' -> ids=[220, 20], toks=['Ġ', '5']
1 -> 16
2 -> 17
3 -> 18
4 -> 19
5 -> 20


In [42]:
### 검증용 / 미리 사전 체크
DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'

def assert_answer_pos_is_digit(
    ds,                    # train_ds or eval_ds (collator 전에 remove_columns 한 ds)
    data_collator,         # DataCollatorForCompletionOnlyLM
    n_samples=1000,
    seed=42,
    pos_from_tail=3,       # -3 규칙
    batch_size=16,
    verbose_fail=5,
):
    rng = random.Random(seed)
    n_samples = min(n_samples, len(ds))
    idxs = rng.sample(range(len(ds)), n_samples)

    fail = 0
    shown = 0

    for start in range(0, n_samples, batch_size):
        batch_idxs = idxs[start:start+batch_size]
        sample_batch = [ds[i] for i in batch_idxs]
        collated = data_collator(sample_batch)

        input_ids = collated["input_ids"]          # (B, L)
        attn = collated["attention_mask"]          # (B, L)
        labels = collated["labels"]                # (B, L)

        real_len = attn.sum(dim=1)                 # (B,)
        pos = real_len - pos_from_tail             # (B,)

        bsz = input_ids.size(0)
        for b in range(bsz):
            p = int(pos[b].item())
            tok = int(input_ids[b, p].item())
            lab = int(labels[b, p].item())

            ok = (tok in DIGIT_IDS) and (lab in DIGIT_IDS)
            if not ok:
                fail += 1
                if shown < verbose_fail:
                    shown += 1
                    print(f"[FAIL] idx={batch_idxs[b]} real_len={int(real_len[b])} pos={p} tok={tok} lab={lab}")
                    # 주변 토큰도 같이 확인
                    left = max(0, p-6)
                    right = min(input_ids.size(1), p+6)
                    print("  decoded window:", tokenizer.decode(input_ids[b, left:right], skip_special_tokens=False))
                    # labels에서 digit 토큰이 아예 있는지 확인
                    digit_mask = torch.isin(labels[b], torch.tensor(DIGIT_IDS, device=labels.device))
                    if digit_mask.any():
                        digit_positions = torch.where(digit_mask)[0].tolist()
                        print("  digit_positions_in_labels:", digit_positions)
                    else:
                        print("  digit_positions_in_labels: NONE")

    assert fail == 0, f"pos=real_len-{pos_from_tail} 규칙이 깨진 샘플이 {fail}개 있습니다."
    print(f"OK: {n_samples} samples passed. (pos=real_len-{pos_from_tail} is digit token)")

In [43]:
# collator 돌리기 전에 id/label 제거한 ds 기준
train_ds = tokenized_dataset["train"].remove_columns(["id", "label"])
eval_ds  = tokenized_dataset["validation"].remove_columns(["id", "label"])

assert_answer_pos_is_digit(train_ds, data_collator, n_samples=1500, pos_from_tail=3)

OK: 1500 samples passed. (pos=real_len-3 is digit token)


In [44]:
DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'

def preprocess_logits_for_metrics(logits, labels, pos_from_tail=4):
    """
    반환: (batch, 5)  -> '1'~'5'에 해당하는 logits만 뽑아서 metrics 단계로 전달
    """
    # Trainer가 (logits, ...) 튜플을 줄 때가 있어서 정리
    if isinstance(logits, tuple):
        logits = logits[0]  # (B, L, V)

    # labels: (B, L), pad/무시 영역은 -100일 가능성이 큼
    # real_len = 마지막으로 labels != -100 인 위치 + 1 로 복원
    labels_t = torch.as_tensor(labels)
    not_ignored = (labels_t != -100)

    # 샘플별로 마지막 not_ignored 위치 찾기
    # (뒤에서부터 True 찾기)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)          # (B,)
    has_any = not_ignored.any(dim=1)                             # (B,)
    # real_len = seq_len - last_true_from_end
    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end

    # 만약 labels가 전부 -100인 샘플이 있으면(비정상) 그냥 seq_len로 처리
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos = (real_len - pos_from_tail).clamp(min=0, max=seq_len-1) # (B,)

    # (B, V)로 해당 위치의 logits만 gather
    logits_t = torch.as_tensor(logits)                           # (B, L, V)
    batch_idx = torch.arange(logits_t.size(0), device=logits_t.device)
    picked = logits_t[batch_idx, pos, :]                         # (B, V)

    # digit ids만 슬라이스 -> (B, 5)
    picked_digits = picked[:, DIGIT_IDS]
    return picked_digits

In [45]:
# ### 실행 하지 않아도 됨
# # 검증용 GPT 생성 함수
# DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'

# @torch.no_grad()
# def check_minus4_vs_minus3(
#     model,
#     ds,                 # train_ds or eval_ds (id/label 제거된 것)
#     data_collator,      # DataCollatorForCompletionOnlyLM
#     n_samples=500,
#     seed=42,
#     batch_size=16,
#     pos_from_tail=3,    # digit 위치가 real_len-3 이라는 네 규칙
#     device=None,
#     verbose=5,
# ):
#     model.eval()
#     if device is None:
#         device = next(model.parameters()).device

#     rng = random.Random(seed)
#     n_samples = min(n_samples, len(ds))
#     idxs = rng.sample(range(len(ds)), n_samples)

#     total = 0
#     hit_m3 = 0  # logits at p (=-3)로 digit 맞춘 횟수
#     hit_m4 = 0  # logits at p-1 (=-4)로 digit 맞춘 횟수

#     shown = 0

#     for s in range(0, n_samples, batch_size):
#         batch_idxs = idxs[s:s+batch_size]
#         batch = [ds[i] for i in batch_idxs]
#         coll = data_collator(batch)

#         input_ids = coll["input_ids"].to(device)         # (B,L)
#         attention_mask = coll["attention_mask"].to(device)
#         labels = coll["labels"].to(device)               # (B,L)

#         # forward
#         out = model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = out.logits                               # (B,L,V)

#         real_len = attention_mask.sum(dim=1)              # (B,)
#         p = (real_len - pos_from_tail).long()             # digit 토큰 위치 (B,)
#         p = torch.clamp(p, min=0, max=input_ids.size(1)-1)

#         bsz = input_ids.size(0)
#         for b in range(bsz):
#             pb = int(p[b].item())
#             if pb == 0:
#                 continue

#             # 정답 digit token id (labels에서 가져오는 게 가장 안전)
#             gold = int(labels[b, pb].item())
#             if gold not in DIGIT_IDS:
#                 # 혹시 label 마스킹/템플릿 변화로 digit이 여기 없으면 스킵
#                 continue

#             # (1) -4 후보: logits[p-1]에서 DIGIT_IDS 중 argmax
#             scores_m4 = logits[b, pb-1, DIGIT_IDS]
#             pred_m4 = DIGIT_IDS[int(torch.argmax(scores_m4).item())]

#             # (2) -3 후보: logits[p]에서 DIGIT_IDS 중 argmax
#             scores_m3 = logits[b, pb, DIGIT_IDS]
#             pred_m3 = DIGIT_IDS[int(torch.argmax(scores_m3).item())]

#             total += 1
#             hit_m4 += (pred_m4 == gold)
#             hit_m3 += (pred_m3 == gold)

#             if shown < verbose and (pred_m4 != gold or pred_m3 != gold):
#                 shown += 1
#                 print(f"[ex] idx={batch_idxs[b]}  real_len={int(real_len[b])}  p(real_len-3)={pb}")
#                 print(f"     gold={gold}({tokenizer.decode([gold])}) | "
#                       f"pred_m4={pred_m4}({tokenizer.decode([pred_m4])}) | "
#                       f"pred_m3={pred_m3}({tokenizer.decode([pred_m3])})")

#     acc_m4 = hit_m4 / max(total, 1)
#     acc_m3 = hit_m3 / max(total, 1)

#     print(f"\nChecked {total} valid samples")
#     print(f"Acc using logits at p-1 (=-4): {acc_m4:.4f}")
#     print(f"Acc using logits at p   (=-3): {acc_m3:.4f}")

#     if acc_m4 > acc_m3:
#         print("=> 결론: digit 예측 위치는 대부분 p-1 (= real_len-4) 쪽이 맞습니다.")
#     else:
#         print("=> 결론: digit 예측 위치가 p (= real_len-3) 쪽일 가능성이 있습니다. (템플릿/라벨 구조 재확인 필요)")


# train_ds = tokenized_dataset["train"].remove_columns(["id", "label"])
# check_minus4_vs_minus3(
#     model=model,
#     ds=train_ds,
#     data_collator=data_collator,
#     n_samples=500,
#     batch_size=8,
#     pos_from_tail=3,
# )

- 근데 이렇게 되면 4,5 선지에 따른 macro-f1이 달라지지 않나...?

In [46]:
def compute_metrics(eval_pred, label_pos_from_tail=3):
    """
    eval_pred:
      - (predictions, label_ids) 튜플 형태가 가장 흔함
      - predictions: preprocess_logits_for_metrics가 반환한 (B, 5)
      - label_ids: (B, L) with -100 ignored
    반환: {"accuracy": ..., "macro_f1": ...}
    """
    if hasattr(eval_pred, "predictions"):
        preds, labels = eval_pred.predictions, eval_pred.label_ids
    else:
        preds, labels = eval_pred

    preds_t = torch.as_tensor(preds)
    pred_cls = torch.argmax(preds_t, dim=-1).cpu().numpy().astype(np.int64)  # (B,)

    labels_t = torch.as_tensor(labels)

    not_ignored = (labels_t != -100)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)
    has_any = not_ignored.any(dim=1)

    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos_label = (real_len - label_pos_from_tail).clamp(min=0, max=seq_len - 1)
    batch_idx = torch.arange(labels_t.size(0), device=labels_t.device)
    gold_tok = labels_t[batch_idx, pos_label].cpu().numpy().astype(np.int64) 

    gold_cls = gold_tok - DIGIT_IDS[0]  

    valid = (gold_cls >= 0) & (gold_cls < 5)
    pred_cls = pred_cls[valid]
    gold_cls = gold_cls[valid]

    acc = (pred_cls == gold_cls).mean() if len(gold_cls) > 0 else 0.0

    f1s = []
    for c in range(5):
        tp = np.sum((pred_cls == c) & (gold_cls == c))
        fp = np.sum((pred_cls == c) & (gold_cls != c))
        fn = np.sum((pred_cls != c) & (gold_cls == c))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
        f1s.append(f1)

    macro_f1 = float(np.mean(f1s)) if len(f1s) > 0 else 0.0

    return {"accuracy": float(acc), "macro_f1": macro_f1}

### Train

In [47]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [48]:
# Trainer 전에는 반드시 지워야 하는것 -> 'input_ids', 'attention_mask'
# 아래 예시처럼
train_dataset = tokenized_dataset["train"].remove_columns(["id", "label"])
eval_dataset = tokenized_dataset["validation"].remove_columns(["id", "label"])

1.	bitsandbytes 로딩 성공(여기서 에러 나면 먼저 해결)  ￼
2.	4bit로 모델 로드
3.	k-bit 학습 준비 + LoRA 어댑터 장착
4.	SFTTrainer에 data_collator/metrics 연결
5.	짧은 max_length(1024) + batch1로 100~200 step만 돌려서 OOM/학습 정상동작 확인
6.	그 다음에 길이 늘리기(1536→2048), grad_accum 조절

In [49]:
print("pad_token:", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos_token:", tokenizer.eos_token, tokenizer.eos_token_id)

# pad_token 없으면 eos로 대체
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# 학습 안정/메모리 옵션
model.config.use_cache = False
model.gradient_checkpointing_enable()

print("use_cache:", model.config.use_cache)
print("grad_ckpt:", model.is_gradient_checkpointing)

pad_token: <|endoftext|> 151643
eos_token: <|im_end|> 151645
use_cache: False
grad_ckpt: True


In [ ]:
MODEL_NAME = "Qwen/Qwen3-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,  # V100이면 fp16 추천
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [51]:
# tokenizer에 pad_token이 이미 있는 상태라면
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [52]:
print("tokenizer.pad_token_id:", tokenizer.pad_token_id)
print("model.config.pad_token_id:", model.config.pad_token_id)
print("gen.pad_token_id:", model.generation_config.pad_token_id)

tokenizer.pad_token_id: 151643
model.config.pad_token_id: 151643
gen.pad_token_id: 151643


In [ ]:
model = prepare_model_for_kbit_training(model)

# Attention proj만
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

# Attention + MLP까지 (성능 더 노리되 trainable 조금 증가)
# "gate_proj", "up_proj", "down_proj" -> FFN
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# target_modules = ["q_proj", "k_proj"]

target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301


In [55]:
training_args = SFTConfig(
    output_dir="../../qwen-sft-results",
    
    # 데이터 및 배치 설정
    num_train_epochs=1,
    max_seq_length=2048,
    packing=False,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    
    #  학습률 및 Optimizer (V100)
    learning_rate=2e-4,
    fp16=True,                  # V100 필수
    bf16=False,
    optim="paged_adamw_32bit",  #  32bit 사용 (안정성) / paged_adamw_8bit -> 더 안된다면 이걸로!
    gradient_checkpointing=True,
    
    # 검증(Eval) 및 저장(Save) 전략 수정
    eval_strategy="steps",      # Epoch 대신 Steps 단위로 검증
    eval_steps=20,              # 50 스텝마다 검증 (데이터 양에 따라 조절하세요)
    
    # 실제 사용 시 주석 해제
    save_strategy="steps",      # 검증 주기와 맞춰서 저장 전략도 steps로
    save_steps=20,              # 50 스텝마다 저장 시도
    
    # 저장 용량 관리
    save_total_limit=2,         # 체크포인트를 딱 2개만 남기고 옛날 건 자동 삭제!
    load_best_model_at_end=True, # 학습 끝나면 "가장 성능 좋았던 모델"을 자동으로 로드
    metric_for_best_model="eval_loss", # 무엇을 기준으로 최고를 뽑을지 (accuracy 추천)
    greater_is_better=False,
    
    # 기타 로깅
    logging_steps=10,
    report_to="none",

    # 시험용이라면???
    # max_steps = 50
)

In [56]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args=training_args,
)

/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [57]:
torch.cuda.empty_cache()

trainer.train()


del model
del trainer
torch.cuda.empty_cache()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss,Accuracy,Macro F1
20,0.075900,0.066489,0.826230,0.789266
40,0.065300,0.053145,0.852459,0.830697
60,0.069400,0.055279,0.849180,0.856548
80,0.071900,0.053688,0.859016,0.869705
100,0.072900,0.054575,0.822951,0.827479
120,0.080300,0.062586,0.852459,0.824387
140,0.075300,0.059562,0.819672,0.814390
160,0.041100,0.049281,0.845902,0.854691
180,0.053000,0.050084,0.865574,0.872092
200,0.047900,0.050142,0.862295,0.867443


- learning rate를 1e-54나 5e-5로 설정하는게 더 좋을듯...

### Inference

In [61]:
# 데이터셋 로드
test_df = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
test_df.info()

# Flatten the JSON dataset
records = []
for _, row in test_df.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
test_df = pd.DataFrame(records)
test_df["choices_len"] = test_df["choices"].apply(len)

test_dataset = Dataset.from_pandas(test_df)

# Model load
MODEL_NAME = "Qwen/Qwen3-8B"
adapter_path = "../../qwen-sft-results/checkpoint-216"# 실제 변경

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(base_model, adapter_path)

def build_test_messages(example):
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
        ]
    }

def to_test_text(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=True, 
    )
    return {"text": text}

test_ds_msg = test_dataset.map(
    build_test_messages,
    batched=False,
    desc="Build messages",
)
test_ds_text = test_ds_msg.map(
    to_test_text,
    batched=False,
    desc="Serialize to text",
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     869 non-null    int64 
 1   id             869 non-null    object
 2   paragraph      869 non-null    object
 3   problems       869 non-null    object
 4   question_plus  44 non-null     object
dtypes: int64(1), object(4)
memory usage: 34.1+ KB


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Build messages:   0%|          | 0/869 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/869 [00:00<?, ? examples/s]

In [65]:
test_ds_text

Dataset({
    features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len', 'messages', 'text'],
    num_rows: 869
})

In [ ]:
sample_size = 100
shuffled_ds = test_ds_text.shuffle(seed=42) 
sample_ds = shuffled_ds.select(range(sample_size))

results = []

print(f"총 {sample_size}개의 샘플에 대해 무작위 테스트를 시작합니다...\n")

model.eval()
with torch.inference_mode():
    for i, ex in enumerate(sample_ds):
        _id = ex["id"]
        text = ex["text"]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
        ).to("cuda")

        # 생성 (Generate)
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,      
            do_sample=False,         # Greedy Search (가장 확률 높은 것 선택)
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        # 입력 토큰 이후의 생성된 부분만 디코딩
        input_len = inputs["input_ids"].shape[-1]
        gen_ids = outputs[0][input_len:]
        gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)

        #  정답 추출 로직 (작성하신 로직 적용)
        try:
            # 앞뒤 공백 제거 후 맨 마지막 글자
            pred = gen_text.strip()[-1] 
            
            # 유효성 검사
            if pred not in ['1', '2', '3', '4', '5']:
                print(f"⚠️ [경고] 이상한 값 추출됨: '{pred}' -> '1'로 대체")
                pred = '1'
        except Exception as e:
            print(f"⚠️ [에러] 파싱 실패 ({e}) -> '1'로 대체")
            pred = '1'

        results.append({
            "id": _id,
            "prediction": pred
        })

        # 6. 눈으로 확인하기 (로그 출력)
        print(f"[{i+1}/{sample_size}] ID: {_id}")
        print(f"▶ Raw Output : {repr(gen_text)}") # repr()을 쓰면 \n이 확인 가능
        print(f"▶ Extracted  : {pred}")
        print("-" * 50)

# 최종 결과 확인
print("\n✅ 테스트 완료!")
print(f"추출된 정답 분포: {[r['prediction'] for r in results]}")

🔍 총 100개의 샘플에 대해 무작위 테스트를 시작합니다...

[1/100] ID: generation-for-nlp-1613
▶ Raw Output : '<think>\n\n</think>\n\n4'
▶ Extracted  : 4
--------------------------------------------------
[2/100] ID: generation-for-nlp-2422
▶ Raw Output : '<think>\n\n</think>\n\n1'
▶ Extracted  : 1
--------------------------------------------------
⚠️ [경고] 이상한 값 추출됨: '지' -> '1'로 대체
[3/100] ID: generation-for-nlp-51
▶ Raw Output : '소의 획일화가 불가피하다고 주장하는 이들의 관점을 비판하며, 그들의 주장이 적절하지 않다고 말하고 있어.\n\n### 정답\n3\n\n### 설명\n3번은 ⓒ를 고려할 때, "획일화된 장소에 식상함을 느낀 사람들이 장소의 선택권을 요구했다는 점"을 근거로 제시하고 있다는 내용을 포함하고 있지만, 지'
▶ Extracted  : 1
--------------------------------------------------
[4/100] ID: generation-for-nlp-2723
▶ Raw Output : '<think>\n\n</think>\n\n4'
▶ Extracted  : 4
--------------------------------------------------
[5/100] ID: generation-for-nlp-990
▶ Raw Output : '<think>\n\n</think>\n\n2'
▶ Extracted  : 2
--------------------------------------------------
[6/100] ID: generation-for-nlp-330
▶ Raw Output : '<think>\n

In [ ]:
import torch
from tqdm import tqdm

infer_results = [] # 변수명 통일 (infer_results 대신 results 사용)

print("안전한 Generate 추론 시작...")

model.eval()
with torch.inference_mode():
    for ex in tqdm(test_ds_text):
        _id = ex["id"]
        text = ex["text"]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=4096,
        ).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=512,      
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        input_len = inputs["input_ids"].shape[-1]
        gen_ids = outputs[0][input_len:]
        gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)

        pred = '1' # 기본값 (최악의 경우 1번 찍기)
        
        # 텍스트 전체에서 뒤집어서(reversed) 검사
        for char in reversed(gen_text):
            if char in ['1', '2', '3', '4', '5']:
                pred = char
                break # 숫자를 찾으면 즉시 중단
        
        # 결과 저장
        infer_results.append({
            "id": _id,
            "answer": pred
        })

print(f" 완료! 총 {len(infer_results)}개 결과 생성됨")

안전한 Generate 추론 시작...


100%|██████████| 869/869 [18:47<00:00,  1.30s/it]

 완료! 총 869개 결과 생성됨


In [78]:
pd.DataFrame(infer_results).to_csv("output.csv", index=False)

In [79]:
pd.DataFrame(infer_results)

,id,prediction
0,generation-for-nlp-0,5
1,generation-for-nlp-1,5
2,generation-for-nlp-2,4
3,generation-for-nlp-3,5
4,generation-for-nlp-4,3
...,...,...
864,generation-for-nlp-1609,1
865,generation-for-nlp-1512,1
866,generation-for-nlp-1382,3
867,generation-for-nlp-702,4


In [ ]:
### 이걸로 해보면...??? ###

DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'
THINK_END_ID = 151668  # </think>

def get_answer_from_logits(outputs, input_len):
    """
    output_scores에서 </think> 이후 첫 digit의 logits 확인
    """
    if not hasattr(outputs, 'scores') or not outputs.scores:
        return None
    
    generated_ids = outputs.sequences[0]  # (total_len,)
    
    # </think> 위치 찾기
    think_end_positions = (generated_ids == THINK_END_ID).nonzero(as_tuple=True)[0]
    
    if len(think_end_positions) == 0:
        # </think>가 없으면 생성된 토큰 중 첫 digit 찾기
        for i in range(input_len, len(generated_ids)):
            token_id = generated_ids[i].item()
            if token_id in DIGIT_IDS:
                step_idx = i - input_len
                if 0 <= step_idx < len(outputs.scores):
                    step_logits = outputs.scores[step_idx][0]  # (V,)
                    digit_logits = step_logits[DIGIT_IDS]  # (5,)
                    return digit_logits.argmax().item() + 1  # 1~5
        return None
    
    # </think> 이후 첫 digit 토큰 찾기
    think_end_pos = think_end_positions[-1].item()
    
    for i in range(think_end_pos + 1, len(generated_ids)):
        token_id = generated_ids[i].item()
        if token_id in DIGIT_IDS:
            # 이 토큰이 생성된 step의 logits 확인
            step_idx = i - input_len
            if 0 <= step_idx < len(outputs.scores):
                step_logits = outputs.scores[step_idx][0]  # (V,)
                digit_logits = step_logits[DIGIT_IDS]  # (5,)
                # 가장 높은 확률의 digit 선택
                predicted = digit_logits.argmax().item() + 1  # 1~5
                return predicted
    
    return None


def parse_pred_fallback(text):
    """
    logits에서 실패 시 텍스트 파싱 (fallback)
    """
    if "</think>" in text:
        after_think = text.split("</think>")[-1]
        for char in after_think:
            if char in ['1', '2', '3', '4', '5']:
                return char
    
    for keyword in ["정답:", "정답은", "Answer:"]:
        if keyword in text:
            after_keyword = text.split(keyword)[-1]
            for char in after_keyword:
                if char in ['1', '2', '3', '4', '5']:
                    return char
    
    for char in reversed(text):
        if char in ['1', '2', '3', '4', '5']:
            return char
    
    return '1'


# ==========================================
# 추론 실행 (개선 버전)
# ==========================================
infer_results = []
logits_success = 0
fallback_used = 0

print("🚀 Logits 기반 추론 시작...")

model.eval()
with torch.inference_mode():
    for ex in tqdm(test_ds_text):
        _id = ex["id"]
        text = ex["text"]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=4096,
        ).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,       
        )

        input_len = inputs["input_ids"].shape[-1]
        
        # 1차 시도: logits 기반
        pred = get_answer_from_logits(outputs, input_len)
        
        if pred is not None:
            logits_success += 1
            pred_str = str(pred)
        else:
            # 2차 시도: 텍스트 파싱
            fallback_used += 1
            gen_ids = outputs.sequences[0][input_len:]
            gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
            pred_str = parse_pred_fallback(gen_text)
        
        infer_results.append({
            "id": _id,
            "answer": pred_str
        })

print(f"✅ 완료! 총 {len(infer_results)}개 결과 생성됨")
print(f"📊 Logits 성공: {logits_success} ({logits_success/len(infer_results)*100:.1f}%)")
print(f"📊 Fallback 사용: {fallback_used} ({fallback_used/len(infer_results)*100:.1f}%)")

🚀 Logits 기반 추론 시작...


100%|██████████| 869/869 [18:50<00:00,  1.30s/it]

✅ 완료! 총 869개 결과 생성됨
📊 Logits 성공: 869 (100.0%)
📊 Fallback 사용: 0 (0.0%)


In [84]:
pd.DataFrame(infer_results).to_csv("output2.csv", index=False)

In [85]:
pd.DataFrame(infer_results)

,id,prediction
0,generation-for-nlp-0,5
1,generation-for-nlp-1,5
2,generation-for-nlp-2,4
3,generation-for-nlp-3,5
4,generation-for-nlp-4,3
...,...,...
864,generation-for-nlp-1609,1
865,generation-for-nlp-1512,1
866,generation-for-nlp-1382,3
867,generation-for-nlp-702,4
